<a href="https://colab.research.google.com/github/imsidharthj/Fastai/blob/main/Fastai_chapter_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 821.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.2 MB/s eta 0:00:00


In [ ]:
!pip install fastkaggle

In [ ]:
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

In [ ]:
import os
from pathlib import Path

comp = 'paddy-disease-classification'
os.environ['KAGGLE_USERNAME'] = 'imsidharthj'
os.environ['KAGGLE_KEY'] = '6a37f58e98f5572fa4f62ca0bf665f5d'
path = setup_comp(comp, install='fastai "timm>=0.6.2.dev0"')

from fastai.vision.all import *
set_seed(42)
tst_files = get_image_files(path/'test_images').sorted()

100%|██████████| 1.02G/1.02G [00:13<00:00, 81.9MB/s]


In [ ]:
!ls {path}

sample_submission.csv  test_images  train.csv  train_images


In [ ]:
# from fastai.vision.all import *
# set_seed(42)
# path.ls()
# df = pd.read_csv(path/'train.csv')
# df.label.value_counts()

df = pd.read_csv(path/'train.csv', index_col='image_id')
df.head()

,count
label,
normal,1764
blast,1738
hispa,1594
dead_heart,1442
tungro,1088
brown_spot,965
downy_mildew,620
bacterial_leaf_blight,479
bacterial_leaf_streak,380


# trn_path = path/'train_images'
# files = get_image_files(trn_path)

# trn_path = path/'train_images'/'bacterial_panicle_blight'

def get_variety(p): return df.loc[p.name, 'variety']

In [ ]:
# img = PILImage.create(files[0])
# print(img.size)
# img.to_thumb(128)


# def train(arch, size, item=Resize(480, method='squish'), accum=1, finetune=True, epochs=12):
#     dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=item,
#         batch_tfms=aug_transforms(size=size, min_scale=0.75), bs=64//accum)
#     cbs = GradientAccumulation(64) if accum else []
#     learn = vision_learner(dls, arch, metrics=error_rate, cbs=cbs).to_fp16()
#     if finetune:
#         learn.fine_tune(epochs, 0.01)
#         return learn.tta(dl=dls.test_dl(tst_files))
#     else:
#         learn.unfreeze()
#         learn.fit_one_cycle(epochs, 0.01)


dls = DataBlock(
    blocks=(ImageBlock,CategoryBlock,CategoryBlock),
    n_inp=1,
    get_items=get_image_files,
    get_y = [parent_label,get_variety],
    splitter=RandomSplitter(0.2, seed=42),
    item_tfms=Resize(192, method='squish'),
    batch_tfms=aug_transforms(size=128, min_scale=0.75)
).dataloaders(trn_path)

In [ ]:
# from fastcore.parallel import *

# def f(o): return PILImage.create(o).size
# sizes = parallel(f, files, n_workers=8)
# pd.Series(sizes).value_counts()

# train('convnext_small_in22k', 128, epochs=1, accum=1, finetune=False)

dls.show_batch(max_n=6)

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name convnext_small_in22k to current convnext_small.fb_in22k.
  model = create_fn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,02:18


In [ ]:
# dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, seed=42,
#     item_tfms=Resize(480, method='squish'),
#     batch_tfms=aug_transforms(size=128, min_scale=0.75))
# dls.show_batch(max_n=6)

# import gc
# def report_gpu():
#     print(torch.cuda.list_gpu_processes())
#     gc.collect()
#     torch.cuda.empty_cache()

dls.show_batch(max_n=6)

In [ ]:
# !pip install timm # install the timm library
# from fastai.vision.all import * # import the fastai vision library
# import timm # import the timm library
# learn = vision_learner(dls, 'resnet26d', metrics=error_rate, path='.').to_fp16()

# report_gpu()

def disease_err(inp,disease,variety): return error_rate(inp,disease)
def disease_loss(inp,disease,variety): return F.cross_entropy(inp,disease)

In [ ]:
# learn.lr_find(suggest_funcs=(valley, slide))

# train('convnext_small_in22k', 128, epochs=1, accum=2, finetune=False)

arch = 'convnext_small_in22k'
learn = vision_learner(dls, arch, loss_func=disease_loss, metrics=disease_err, n_out=10).to_fp16()
lr = 0.01

epoch,train_loss,valid_loss,error_rate,time
0,0.000000,0.000000,0.000000,02:21


In [ ]:
# learn.fine_tune(3, 0.01)

# train('convnext_small_in22k', 128, epochs=1, accum=4, finetune=False)
# report_gpu()

learn = vision_learner(dls, arch, n_out=20).to_fp16()

epoch,train_loss,valid_loss,error_rate,time
0,1.735566,1.218735,0.404133,01:31


epoch,train_loss,valid_loss,error_rate,time
0,1.093771,0.824293,0.275348,01:28
1,0.768049,0.497752,0.162902,01:26
2,0.536771,0.397662,0.120615,01:27


In [ ]:
# ss = pd.read_csv(path/'sample_submission.csv')
# ss

# train('convnext_large_in22k', 224, epochs=1, accum=2, finetune=False)

def disease_loss(inp,disease,variety): return F.cross_entropy(inp[:,:10],disease)

/usr/local/lib/python3.10/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name convnext_large_in22k to current convnext_large.fb_in22k.
  model = create_fn(


model.safetensors:   0%|          | 0.00/919M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time


In [ ]:
# tst_files = get_image_files(path/'test_images').sorted()
# tst_dl = dls.test_dl(tst_files)

# train('convnext_large_in22k', (320,240), epochs=1, accum=2, finetune=False)
# report_gpu()

def combine_loss(inp,disease,variety): return disease_loss(inp,disease,variety)+variety_loss(inp,disease,variety)

In [ ]:
# probs,_,idxs = learn.get_preds(dl=tst_dl, with_decoded=True)
# idxs

# train('vit_large_patch16_224', 224, epochs=1, accum=2, finetune=False)
# report_gpu()

def disease_err(inp,disease,variety): return error_rate(inp[:,:10],disease)
def variety_err(inp,disease,variety): return error_rate(inp[:,10:],variety)

err_metrics = (disease_err,variety_err)

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


tensor([7, 8, 7,  ..., 8, 1, 5])

In [ ]:
# dls.vocab

# train('swinv2_large_window12_192_22k', 192, epochs=1, accum=2, finetune=False)
# report_gpu()

all_metrics = err_metrics+(disease_loss,variety_loss)

['bacterial_leaf_blight', 'bacterial_leaf_streak', 'bacterial_panicle_blight', 'blast', 'brown_spot', 'dead_heart', 'downy_mildew', 'hispa', 'normal', 'tungro']

In [ ]:
# mapping = dict(enumerate(dls.vocab))
# results = pd.Series(idxs.numpy(), name="idxs").map(mapping)
# results

# train('swin_large_patch4_window7_224', 224, epochs=1, accum=2, finetune=False)
# report_gpu()

learn = vision_learner(dls, arch, loss_func=combine_loss, metrics=all_metrics, n_out=20).to_fp16()

,idxs
0,hispa
1,normal
2,hispa
3,blast
4,blast
...,...
3464,dead_heart
3465,hispa
3466,normal
3467,bacterial_leaf_streak


In [ ]:
# ss['label'] = results
# ss.to_csv('subm.csv', index=False)
# !head subm.csv

# res = 640,480

learn.fine_tune(5, lr)

image_id,label
200001.jpg,hispa
200002.jpg,normal
200003.jpg,hispa
200004.jpg,blast
200005.jpg,blast
200006.jpg,hispa
200007.jpg,dead_heart
200008.jpg,brown_spot
200009.jpg,hispa


In [ ]:
# if not iskaggle:
#     from kaggle import api
#     api.competition_submit_cli('subm.csv', 'initial rn26d 128px', comp)

models = {
    'convnext_large_in22k': {
        (Resize(res), (320,224)),
    }, 'vit_large_patch16_224': {
        (Resize(480, method='squish'), 224),
        (Resize(res), 224),
    }, 'swinv2_large_window12_192_22k': {
        (Resize(480, method='squish'), 192),
        (Resize(res), 192),
    }, 'swin_large_patch4_window7_224': {
        (Resize(res), 224),
    }
}

100%|██████████| 69.8k/69.8k [00:00<00:00, 116kB/s]


In [ ]:
# if not iskaggle:
#     push_notebook('jhoward', 'Fastai_chapter_6',
#                   title='First Steps: Road to the Top, Part 1',
#                   file='Fastai_chapter_6.ipynb',
#                   competition=comp, private=False, gpu=True)

trn_path = path/'train_images'

In [ ]:
tta_res = []

for arch,details in models.items():
    for item,size in details:
        print('---',arch)
        print(size)
        print(item.name)
        tta_res.append(train(arch, size, item=item, accum=2)) #, epochs=1))
        gc.collect()
        torch.cuda.empty_cache()

In [ ]:
save_pickle('tta_res.pkl', tta_res)

In [ ]:
tta_prs = first(zip(*tta_res))

In [ ]:
tta_prs += tta_prs[1:3]

In [ ]:
avg_pr = torch.stack(tta_prs).mean(0)
avg_pr.shape

In [ ]:
dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=Resize(480, method='squish'),
    batch_tfms=aug_transforms(size=224, min_scale=0.75))

In [ ]:
idxs = avg_pr.argmax(dim=1)
vocab = np.array(dls.vocab)
ss = pd.read_csv(path/'sample_submission.csv')
ss['label'] = vocab[idxs]
ss.to_csv('subm.csv', index=False)

In [ ]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('subm.csv', 'part 3 v2', comp)

In [ ]:
if not iskaggle:
    push_notebook('jhoward', 'scaling-up-road-to-the-top-part-3',
                  title='Scaling Up: Road to the Top, Part 3',
                  file='10-scaling-up-road-to-the-top-part-3.ipynb',
                  competition=comp, private=False, gpu=True)